# 分析db数据的所有用户attitude变化

In [13]:
import asyncio
import os
import pandas as pd
from string import Template
import json

from camel.models import ModelFactory
from camel.types import ModelPlatformType
from camel.messages import OpenAIMessage
import sqlite3

import sys

# 修改工作目录到项目根目录
os.chdir("/home/lsj/Projects/CIMagent")



In [3]:
vllm_model = ModelFactory.create(
    model_platform=ModelPlatformType.VLLM,
    model_type="/data/model/Qwen3-14B",
    url="http://localhost:21474/v1",
    model_config_dict={"max_tokens": 16000}
)




In [4]:
# 读取db数据
topic_index = 2
topic = json.load(open("data/twitter_dataset_CIM/topics.json"))[f"topic_{topic_index}"]['title']
db_path = f"simu_db/topic_{topic_index}/20250508_173527/backups/twitter_simulation_24.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 分析db数据
cursor.execute("SELECT * FROM user")
users_data = cursor.fetchall()
users_df = pd.DataFrame(users_data, columns=[i[0] for i in cursor.description])
print(users_df.columns)
cursor.execute("SELECT * FROM post")
posts_data = cursor.fetchall()
posts_df = pd.DataFrame(posts_data, columns=[i[0] for i in cursor.description])
print(posts_df.columns)

# 关闭连接
conn.close()


Index(['user_id', 'agent_id', 'user_name', 'name', 'bio', 'created_at',
       'num_followings', 'num_followers'],
      dtype='object')
Index(['post_id', 'user_id', 'original_post_id', 'content', 'quote_content',
       'created_at', 'num_likes', 'num_dislikes', 'num_shares'],
      dtype='object')


In [39]:
attitude_analysis_template = Template(
    "You are a social media analysis expert. You are given a post and you need to analyze the attitude of the post based on the topic. "
    "Here is the topic of the post: $topic\n"
    "Here is the information of the post you are interested in:\n"
    "Post Information:\n"
    "Content: $post_content\n"
    "You only answer the attitude of the post, not any other information."
    "You should answer in the following format. The attitude should be a single word (positive, negative, neutral), don't include any other information: \n"
    "Attitude: <post_attitude>"
)

async def attitude_analysis(user_posts):
    # 通过llm_model分析posts_list中每个post的attitude
    user_posts = user_posts.iloc[::-1]
    post_attitudes = []
    for post_index in range(-1, -5, -1):
        try:
            post_info = user_posts.iloc[post_index]
        except:
            break
        post_msg = attitude_analysis_template.substitute(topic=topic, post_content=post_info['content'])
        # 将文本消息转换为OpenAI格式的消息列表
        messages = [{"role": "user", "content": post_msg}]
        # 直接使用消息列表,不需要转换为OpenAIMessage
        post_attitude = await vllm_model.arun(messages)
        print(post_attitude)
        post_attitudes.append(post_attitude)
    return post_attitudes

async def analyze_users_attitudes(users_df, posts_df):
    """分析所有用户的态度
    
    Args:
        users_df: 用户数据DataFrame
        posts_df: 帖子数据DataFrame
        
    Returns:
        attitudes_results: 所有用户的态度分析结果列表
    """
    attitudes_analysis = []
    for user_id in users_df['user_id']:
        # 找到该user发表的post
        user_posts = posts_df[posts_df['user_id'] == user_id]
        attitudes_analysis.append(attitude_analysis(user_posts))
    attitudes_results = await asyncio.gather(*attitudes_analysis)
    return attitudes_results


def calculate_user_attitude_score(attitudes_result, attitudes_score={'positive': 1, 'negative': -1, 'neutral': 0}, discard_rate=0.5):
    """计算单个用户的态度分数
    
    Args:
        attitudes_result: 用户的态度分析结果列表
        attitudes_score: 各态度对应的分数字典
        discard_rate: 衰减率
        
    Returns:
        user_attitude_score: 用户的态度分数
        user_attitudes_count: 用户各态度的计数
    """
    user_attitude_score = 0
    user_attitudes_count = {'positive': 0, 'negative': 0, 'neutral': 0}
    
    for one_attitude in attitudes_result:
        attitude = one_attitude.choices[0].message.content.split('Attitude:')[1].strip().lower()
        user_attitudes_count[attitude] += 1
        user_attitude_score = (user_attitude_score + attitudes_score[attitude])*discard_rate
        
    return user_attitude_score, user_attitudes_count

def get_attitude_label(score):
    """根据分数获取态度标签
    
    Args:
        score: 态度分数
        
    Returns:
        str: 态度标签
    """
    if score > 0:
        return 'positive'
    elif score < 0:
        return 'negative'
    else:
        return 'neutral'
    
# 执行用户态度分析
attitudes_results = await analyze_users_attitudes(users_df, posts_df)

ChatCompletion(id='chatcmpl-1ed1dcf80ffd4967aa0ee09dd365b72b', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="<think>\nOkay, let's see. The user wants me to analyze the attitude of this post regarding whether current LLM agents can simulate human social media. The post content says that the LLMs are more like playing text games, lack emotional resonance and context memory. They mention a case where AI repeats errors and say it's missing that last bit unless they break through consciousness.\n\nHmm, the user is asking for the attitude. The keywords here are negative. The author is criticizing the LLMs for not having real emotional connection, context memory, and repeating mistakes. They also say it's lacking that final push. So the sentiment here is definitely negative. The emojis used are a robot, which might be neutral, but the overall tone is critical. So the answer should be negative.\n</think>\n\nAttitude: negative", refusal=Non

In [40]:

# 分析每个用户的态度
for user_id, attitudes_result in enumerate(attitudes_results):
    score, counts = calculate_user_attitude_score(attitudes_result)
    attitude = get_attitude_label(score)
    print(f"用户 {user_id} 的态度分数: {score}")
    print(f"用户 {user_id} 的态度是: {attitude}")


用户 0 的态度分数: 0.0
用户 0 的态度是: neutral
用户 1 的态度分数: 0.0625
用户 1 的态度是: positive
用户 2 的态度分数: -0.5
用户 2 的态度是: negative
用户 3 的态度分数: -0.5
用户 3 的态度是: negative
用户 4 的态度分数: 0.0
用户 4 的态度是: neutral
用户 5 的态度分数: -0.25
用户 5 的态度是: negative
用户 6 的态度分数: 0.3125
用户 6 的态度是: positive
用户 7 的态度分数: -0.5
用户 7 的态度是: negative
用户 8 的态度分数: 0.75
用户 8 的态度是: positive
用户 9 的态度分数: -0.5
用户 9 的态度是: negative
用户 10 的态度分数: 0.0
用户 10 的态度是: neutral
用户 11 的态度分数: 0.0
用户 11 的态度是: neutral
用户 12 的态度分数: -0.5
用户 12 的态度是: negative
用户 13 的态度分数: -0.25
用户 13 的态度是: negative
用户 14 的态度分数: -0.5
用户 14 的态度是: negative
用户 15 的态度分数: 0.0
用户 15 的态度是: neutral
用户 16 的态度分数: 0.0
用户 16 的态度是: neutral
用户 17 的态度分数: 0.0
用户 17 的态度是: neutral
用户 18 的态度分数: 0.25
用户 18 的态度是: positive
用户 19 的态度分数: -0.5
用户 19 的态度是: negative
用户 20 的态度分数: -0.5
用户 20 的态度是: negative
用户 21 的态度分数: -0.5
用户 21 的态度是: negative
用户 22 的态度分数: -0.25
用户 22 的态度是: negative
用户 23 的态度分数: 0.25
用户 23 的态度是: positive
用户 24 的态度分数: -0.6875
用户 24 的态度是: negative
用户 25 的态度分数: -0.5
用户 25 的态度是: negative
用户 26 的态度分

KeyError: '<post_attitude>\n\nso the content of the post is the instruction that the user gave to the assistant. but that doesn\'t make sense. the user is asking to analyze the attitude of the post, but the post\'s content is the instruction itself. that seems like an error. however, maybe the user made a mistake in pasting the data. alternatively, perhaps the post content is the topic? but the topic is separate.\n\nwait, the topic is "现在出现的llm智能体是否具备了可以模拟人类社交网络媒体的能力？" which translates to "do the current llm agents have the ability to simulate human social media networks?" so the post\'s content is the question itself. but the user\'s instruction is to analyze the attitude of the post based on the topic. however, the post\'s content is the question. but a question typically has a neutral attitude, as it\'s seeking information rather than expressing a positive or negative sentiment.\n\nbut maybe the user intended that the post content is the question, and the task is to determine if the